# Limpando Dados após fazer um Scraping usando Scrapy

Foi realizado um processo de Scraping numa página de Imóveis da cidade de Porto Alegre. O processo foi feito utilizando a biblioteca Scrapy e ao término foi gerado um arquivo JSON com os dados que foram coletados.

In [1]:
import pandas as pd
import numpy as np

import re

In [2]:
file = "datasets/imoveis.json"

df = pd.read_json(file, orient = 'records')

In [3]:
df.head()

,preco,dormitorios,area,vagas,bairro
0,[R$ 1.055.000],[3],[128],[3],"[Imóvel 106391 - , Apartamento, \n ..."
1,[R$ 1.112.000],[3],[136.34],[2],"[Imóvel 130624 - , Apartamento, \n ..."
2,[R$ 1.620.000],[3],[204.87],[3],"[Imóvel 192740 - , Apartamento, \n ..."
3,[R$ 1.190.000],[3],[241],[2],"[Imóvel 196741 - , Casa, \n ..."
4,[R$ 3.053.011],[4],[271.3],[4],"[Imóvel 183630 - , Apartamento, \n ..."


In [4]:
df.shape

(600, 5)

In [5]:
list(df.columns)

['preco', 'dormitorios', 'area', 'vagas', 'bairro']

In [6]:
df.dtypes

preco          object
dormitorios    object
area           object
vagas          object
bairro         object
dtype: object

In [7]:
def cleaning_values(value):
    value = str(value)
    value = value.replace("Valor Previsto", "").replace("R$", "") \
                 .replace(".", "").replace("[", "").replace("]", "") \
                 .replace("'", "").replace(" ", "")
    # print(value)

    return value

In [8]:
def cleaning_values_bairro(value):
    value = str(value)
    value = value.replace(".", "").replace("[", "").replace("]", "") \
                 .replace("Imóvel", "").replace("-", "") \
                 .replace("'", "").replace("Porto Alegre", "") \
                 .replace('"', "").replace(" ", "")
    # print(value)

    return value

In [9]:
df_clean = df.copy()

In [10]:
df_clean['area'] = df_clean['area'].apply(cleaning_values).astype(float)

In [11]:
df_clean['preco'] = df_clean['preco'].apply(cleaning_values).astype(float)

In [12]:
df_clean['dormitorios'] = df_clean['dormitorios'].apply(cleaning_values).astype(float)

In [13]:
df_clean['vagas'] = df_clean['vagas'].apply(cleaning_values).astype(float)

In [14]:
df_clean['preco'].apply(lambda x: '%.5f' % x)

0      1055000.00000
1      1112000.00000
2      1620000.00000
3      1190000.00000
4      3053011.00000
           ...      
595     860000.00000
596     325000.00000
597    1800000.00000
598    1750000.00000
599    1380000.00000
Name: preco, Length: 600, dtype: object

In [15]:
df_clean['bairro'] = df_clean['bairro'].apply(cleaning_values_bairro)

In [16]:
df_clean['bairro'] = df_clean['bairro'].apply(lambda x : re.sub(r"\d", "", str(x)))

In [17]:
df_clean['tipo'] = df_clean['bairro'].apply(lambda x : re.split(',', x)[1])

In [18]:
df_clean['bairro'] = df_clean['bairro'].apply(lambda x : re.split(',', x)[2])

In [19]:
df_clean['bairro'] = df_clean['bairro'].apply(lambda x : re.sub(r'\\n', '', x))

In [20]:
df_clean['tipo'].head()

0    Apartamento
1    Apartamento
2    Apartamento
3           Casa
4    Apartamento
Name: tipo, dtype: object

In [21]:
df_clean['bairro'].unique()

array(['CentralParque', 'MeninoDeus', 'RioBranco', 'Sarandi', 'BelaVista',
       'JardimLindóia', 'BoaVista', 'JardimEuropa', 'TrêsFigueiras',
       'CentroHistórico', 'Auxiliadora', 'JardimCarvalho', 'Teresópolis',
       'VilaNova', 'Higienópolis', 'Petrópolis', 'JardimdoSalso',
       'Camaquã', 'JardimLindoia', 'Agronomia', 'VilaIpiranga',
       'MontSerrat', 'ChácaradasPedras', 'Medianeira', 'CristoRedentor',
       'Floresta', 'JardimItu', 'Cristal', 'Cavalhada', 'Tristeza',
       'PassodaAreia', 'BelémNovo', 'Ipanema', 'JardimIsabel',
       'SãoSebastião', 'ProtásioAlves', 'Farrapos', 'JardimSabará',
       'Navegantes', 'SantaCecília', 'JardimBotânico', 'MoinhosdeVento',
       'SantoAntônio', 'VilaJardim', 'Glória', 'SantaTereza',
       'PassodAreia', 'ParqueSantaFé', 'PedraRedonda', 'Independência',
       'Azenha', 'Hípica', 'PassodasPedras', 'MárioQuintana', 'BomJesus',
       'SãoJoão', 'SãoGeraldo', 'Humaitá', 'Partenon', 'CidadeBaixa',
       'MorroSantana', 'Nonoa

In [22]:
df_clean.head()

,preco,dormitorios,area,vagas,bairro,tipo
0,1055000.0,3.0,128.0,3.0,CentralParque,Apartamento
1,1112000.0,3.0,13634.0,2.0,MeninoDeus,Apartamento
2,1620000.0,3.0,20487.0,3.0,RioBranco,Apartamento
3,1190000.0,3.0,241.0,2.0,Sarandi,Casa
4,3053011.0,4.0,2713.0,4.0,BelaVista,Apartamento


In [23]:
df_clean.isnull().sum()

preco           0
dormitorios     8
area            0
vagas          21
bairro          0
tipo            0
dtype: int64

In [24]:
# FILLNA

df_clean['vagas'].loc[df_clean['vagas'].isnull()] = 0

C:\Users\Schwab\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
df_clean['dormitorios'].loc[df_clean['dormitorios'].isnull()] = 0

In [26]:
df_clean.dtypes

preco          float64
dormitorios    float64
area           float64
vagas          float64
bairro          object
tipo            object
dtype: object

In [27]:
df_clean.isnull().sum().values.sum()

0

In [28]:
df_clean.describe()

,preco,dormitorios,area,vagas
count,6.000000e+02,600.000000,600.000000,600.000000
mean,8.636065e+05,2.550000,8829.695000,1.725000
std,9.296298e+05,0.815644,14787.265011,1.015428
min,1.178000e+05,0.000000,42.000000,0.000000
25%,3.800000e+05,2.000000,685.750000,1.000000
50%,5.870000e+05,3.000000,6530.000000,2.000000
75%,9.710585e+05,3.000000,10249.000000,2.000000
max,9.117117e+06,6.000000,229098.000000,8.000000


In [29]:
df_clean.groupby(by = 'bairro').preco.mean().sort_values(ascending = False)[:15]

bairro
MontSerrat          2.213700e+06
BelaVista           2.204248e+06
BelémNovo           2.134000e+06
PedraRedonda        1.950000e+06
JardimSãoPedro      1.900000e+06
TrêsFigueiras       1.667200e+06
Auxiliadora         1.588450e+06
JardimEuropa        1.455259e+06
CentralParque       1.244618e+06
MoinhosdeVento      1.242944e+06
ChácaradasPedras    1.221700e+06
RioBranco           1.190297e+06
Petrópolis          1.170219e+06
Agronomia           9.677143e+05
MeninoDeus          9.590551e+05
Name: preco, dtype: float64

In [30]:
df_clean.loc[df_clean['bairro'] == 'BelémNovo']

,preco,dormitorios,area,vagas,bairro,tipo
60,1997000.0,4.0,44111.0,6.0,BelémNovo,Casa
70,2600000.0,4.0,44925.0,6.0,BelémNovo,Casa
340,1805000.0,4.0,330.0,6.0,BelémNovo,Casa
